# Hacker News Project

[Hacker News](https://en.wikipedia.org/wiki/Hacker_News) is a social news website focusing on computer science and entrepreneurship. The word hacker in "Hacker News" is used in its original meaning and refers to the hacker culture which consists of people who enjoy tinkering with technology. This project looks into the posts on the Hacker News website.

The purpose of this project is to answer the following:

1. Determine which type of post, 'Ask HN' or 'Show HN', recieves more comments.
2. Do posts created at a certain time receive more comments on average?

In [1]:
import pandas as pd
import os
import numpy as np
import csv

In [2]:
print(os.getcwd())
os.listdir()

C:\Users\gfulham\Dataquest_JPNBs\1_intro_exercises


['.ipynb_checkpoints',
 '1.1_Basics_Project.ipynb',
 '1.2 Intermediate Basics Hacker News.ipynb',
 'HN_posts_year_to_Sep_26_2016.csv']

In [3]:
# Reading in data as a list of lists instead of a df. Only since beginner course and working with lists
file = open("HN_posts_year_to_Sep_26_2016.csv", encoding="utf8") # Had to specify because of coding error. 
hn = list(csv.reader(file))
hn[:5] #Quick look

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12579008',
  'You have two days to comment if you want stem cells to be classified as your own',
  'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018',
  '1',
  '0',
  'altstar',
  '9/26/2016 3:26'],
 ['12579005',
  'SQLAR  the SQLite Archiver',
  'https://www.sqlite.org/sqlar/doc/trunk/README.md',
  '1',
  '0',
  'blacksqr',
  '9/26/2016 3:24'],
 ['12578997',
  'What if we just printed a flatscreen television on the side of our boxes?',
  'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43',
  '1',
  '0',
  'pavel_lishin',
  '9/26/2016 3:19'],
 ['12578989',
  'algorithmic music',
  'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext',
  '1',
  '0',
  'poindontcare',
  '9/26/2016 3:16']]

In [4]:
#Get header row
headers = hn[0]
print(headers) # Looking at the header variable
hn = hn[1:] # Removing the header from the dataset
hn[:5] #Making sure the dataset has the header removed. 

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12579008',
  'You have two days to comment if you want stem cells to be classified as your own',
  'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018',
  '1',
  '0',
  'altstar',
  '9/26/2016 3:26'],
 ['12579005',
  'SQLAR  the SQLite Archiver',
  'https://www.sqlite.org/sqlar/doc/trunk/README.md',
  '1',
  '0',
  'blacksqr',
  '9/26/2016 3:24'],
 ['12578997',
  'What if we just printed a flatscreen television on the side of our boxes?',
  'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43',
  '1',
  '0',
  'pavel_lishin',
  '9/26/2016 3:19'],
 ['12578989',
  'algorithmic music',
  'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext',
  '1',
  '0',
  'poindontcare',
  '9/26/2016 3:16'],
 ['12578979',
  'How the Data Vault Enables the Next-Gen Data Warehouse and Data Lake',
  'https://www.talend.com/blog/2016/05/12/talend-and-Â\x93the-data-vaultÂ\x94',
  '1',
  '0',
  'markgainor1',
  '9/26/2016 3:14']]

In [8]:
#Create 3 lists outside the loop
ask_hn = []
show_hn = []
other_hn = []

#loop through rows in hn
for each in hn:
    title = each[1]
    title = title.lower()
    if title.startswith('ask hn'):
        ask_hn.append(each)
    elif title.startswith('show hn'):
        show_hn.append(each)
    else:
        other_hn.append(each)

#Check length of each list
print(len(ask_hn))
print(len(show_hn))
print(len(other_hn))

9139
10158
273822


In [34]:
#Find the total and average amount of comments for the Ask_hn and Show_hn lists. 

#Find total comments for ask_hn
total_ask_comments = 0
for each in ask_hn:
    comment_count = int(each[4])
    total_ask_comments += comment_count

#Find total commments for show_hn
total_show_comments = 0
for each in show_hn:
    comment_count = int(each[4])
    total_show_comments += comment_count
    
#Find avg comments
avg_ask_comments = total_ask_comments / len(ask_hn)
avg_show_comments = total_show_comments / len(show_hn)

print('Ask HN Total comments= ', total_ask_comments)
print('Ask HN Average comments= {}'.format(avg_ask_comments))
print('Show HN Total comments = ', total_show_comments)
print('Show HN Average comments = {}'.format(avg_show_comments))

Ask HN Total comments=  94986
Ask HN Average comments= 10.393478498741656
Show HN Total comments =  49633
Show HN Average comments = 4.886099625910612


The data shows there are overall more comments on 'Ask HN' posts, as well as more comments on average from these posts. <br>
Now lets examine if there is a specific time of day that these posts receive more comments. 

In [65]:
#Count the ask posts grouped by each hour of the day 
#Avg the ask posts grouped by each hour of the day
#Use datetime.strptime()
ask_hn[:1]
import datetime as dt 
result_list = []
#iterate ask_posts and append result_list with num comments
for each in ask_hn:
    created_at = each[6]
    num_comments = int(each[4])
    
    result_list.append([created_at,num_comments])
    
print(result_list[:2])
    
#create two dictionaries posts_by_hour and comments_by_hour
posts_by_hour = {} #To see at what time posts are posted, grouped by hour of the day
comments_by_hour = {} #To see how many comments are posted on posts grouped by the hour of the day posted
date_format = '%m/%d/%Y %H:%M'
    
#loop through result_list and extract the hour from the date, being the first element of the row
for each in result_list:
    date = each[0] #put date-string into a variable
    comment = each[1] 
    hour = dt.datetime.strptime(date, date_format).strftime('%H') #strip the hour from the string
    if hour not in posts_by_hour:
        posts_by_hour[hour] = 1
        comments_by_hour[hour] = comment
    else: 
        posts_by_hour[hour] += 1
        comments_by_hour[hour] += comment


comments_by_hour

[['9/26/2016 2:53', 7], ['9/26/2016 1:17', 3]]


{'02': 2996,
 '01': 2089,
 '22': 3372,
 '21': 4500,
 '19': 3954,
 '17': 5547,
 '15': 18525,
 '14': 4972,
 '13': 7245,
 '11': 2797,
 '10': 3013,
 '09': 1477,
 '07': 1585,
 '03': 2154,
 '23': 2297,
 '20': 4462,
 '16': 4466,
 '08': 2362,
 '00': 2277,
 '18': 4877,
 '12': 4234,
 '04': 2360,
 '06': 1587,
 '05': 1838}

## Calculating the Avg Num of Comments on Posts by Hour

In [71]:
# create empty list to store the avgerages
avg_by_hour = []

#Loop over comments_by_hour list. Append the key(hour) and the value(total_comments)/value(total posts) 
for each in comments_by_hour:
    avg_by_hour.append([each,comments_by_hour[each] / posts_by_hour[each]])
    
avg_by_hour

[['02', 11.137546468401487],
 ['01', 7.407801418439717],
 ['22', 8.804177545691905],
 ['21', 8.687258687258687],
 ['19', 7.163043478260869],
 ['17', 9.449744463373083],
 ['15', 28.676470588235293],
 ['14', 9.692007797270955],
 ['13', 16.31756756756757],
 ['11', 8.96474358974359],
 ['10', 10.684397163120567],
 ['09', 6.653153153153153],
 ['07', 7.013274336283186],
 ['03', 7.948339483394834],
 ['23', 6.696793002915452],
 ['20', 8.749019607843136],
 ['16', 7.713298791018998],
 ['08', 9.190661478599221],
 ['00', 7.5647840531561465],
 ['18', 7.94299674267101],
 ['12', 12.380116959064328],
 ['04', 9.7119341563786],
 ['06', 6.782051282051282],
 ['05', 8.794258373205741]]

### Sort the values

In [85]:
swap_avg_by_hour = []
for each in avg_by_hour:
    swap_avg_by_hour.append([each[1],each[0]])


sorted_swap = sorted(swap_avg_by_hour, reverse = True)
print(sorted_swap[0])
print('Top 5 Hours for Ask Posts Comments')
for avg, hr in sorted_swap[:5]:
     print('{}:{:.2f} average comments per post.'.format(dt.datetime.strptime(hr,'%H').strftime('%H:%M'),avg))
    

[28.676470588235293, '15']
Top 5 Hours for Ask Posts Comments
15:00:28.68 average comments per post.
13:00:16.32 average comments per post.
12:00:12.38 average comments per post.
02:00:11.14 average comments per post.
10:00:10.68 average comments per post.


The hour that receives the most comments per post on average in 15:00 with an average of 23.68 comments per post. According the to [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home) this translates to 3:00 PM EST. 